In [1]:
import pandas as pd
import numpy as np
import re
import random

In [2]:
data_location = 'https://github.com/DHDSDesafios/DH_DS_desafio_2_properati/raw/master/data/properatid2caba.zip'
data = pd.read_csv(data_location, compression='zip', encoding="utf-8")

In [3]:
data.sample(10)

,Id_caso,property_type,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,price,currency,...,surface_total_in_m2,surface_covered_in_m2,cochera,piscina,parrilla,baulera,balcon,terraza,jardin,lavadero
13395,55143,apartment,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,"-34.572324,-58.4741025",-34.572324,-58.474103,92000.0,USD,...,38.0,34.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
24871,101138,apartment,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,"-34.5768892189,-58.4234788852",-34.576889,-58.423479,849000.0,USD,...,145.0,145.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3482,17109,apartment,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,"-34.6065929,-58.4429909",-34.606593,-58.442991,170000.0,USD,...,75.0,72.0,NaN,NaN,NaN,NaN,balcon,NaN,NaN,NaN
17012,71315,apartment,|Argentina|Capital Federal|Constitución|,Argentina,Capital Federal,"-34.6236205012,-58.3796860308",-34.623621,-58.379686,71250.0,USD,...,31.0,27.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
11249,46791,PH,|Argentina|Capital Federal|San Telmo|,Argentina,Capital Federal,"-34.6256749,-58.3780604",-34.625675,-58.378060,260000.0,USD,...,183.0,183.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0
19356,77989,apartment,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,NaN,NaN,NaN,90000.0,USD,...,37.0,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2363,11715,apartment,|Argentina|Capital Federal|Congreso|,Argentina,Capital Federal,"-34.6056415854,-58.391482611",-34.605642,-58.391483,53000.0,USD,...,23.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27800,116589,apartment,|Argentina|Capital Federal|Nuñez|,Argentina,Capital Federal,NaN,NaN,NaN,238000.0,USD,...,82.0,71.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
20864,83819,apartment,|Argentina|Capital Federal|Flores|,Argentina,Capital Federal,"-34.6297991751,-58.4638373591",-34.629799,-58.463837,165000.0,USD,...,71.0,68.0,1.0,NaN,NaN,NaN,balcon,NaN,NaN,1.0
998,6369,apartment,|Argentina|Capital Federal|Parque Centenario|,Argentina,Capital Federal,"-34.60695,-58.43924",-34.606950,-58.439240,340000.0,USD,...,20.0,284.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.loc[:, ["property_type", "place_name", "surface_total_in_m2", "surface_covered_in_m2", "price_usd_per_m2"]].corr()

,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2
surface_total_in_m2,1.000000,0.658421,-0.014624
surface_covered_in_m2,0.658421,1.000000,-0.047114
price_usd_per_m2,-0.014624,-0.047114,1.000000


In [3]:
def remove_outlier(df_in, qt1, qt3, iqrout, col_name, mask1, var1, mask2=None, var2=None):
    '''
    Calculo de outliers

    Parameters
    ----------
    df_in : dataFrame
        dataFrame utilizado para extraer variables de filtro y mascaras.
    qt1, qt3: float
        Cuantil inferior y superior (0 a 1).
    iqrout: float
        Margen exterior a iqr (1.5 o 3 recomendado)
    var1, var2: str
        Nombre de columnas de dataFrame ingresado como parametro para aplicar filtro sobre los datos.
    mask1, mask2: str
        Valores para utilizar con las columnas de filtro para generar mascaras booleanas.
    col_name: str
        Variable a aplicar los outliers.
    
    Output
    ------
    dataFrame filtrado por parametros con los outliers aplicados.

    Example
    ----------
    remove_outlier(data, 0.25, 0.75, 1.5, 'price_usd_per_m2', 'Belgrano', 'place_name', 'PH', 'property_type')
    
    '''
    
    if (mask2 is None and var2 is not None) | (mask2 is not None and var2 is None):
        return 'you must set second mask with second variable'
    elif mask2 is None and var2 is None:
        var_mask = df_in[var1] == mask1
    else:
        var_mask = (df_in[var1] == mask1) & (df_in[var2] == mask2)
        
    q1 = df_in.loc[var_mask, col_name].quantile(qt1)
    q3 = df_in.loc[var_mask, col_name].quantile(qt3)
    
    iqr = q3-q1 #Interquartile range
    
    fence_low  = q1-iqrout*iqr
    fence_high = q3+iqrout*iqr
    
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high) & var_mask]
    
    return df_out

In [4]:
df_final = pd.DataFrame()
place_name_list = data.place_name.unique()
prop_type_list = data.property_type.unique()
for pn in place_name_list:
    for pt in prop_type_list:
        df_temp = remove_outlier(data, 0.25, 0.75, 1.5, 'price_usd_per_m2', pn, 'place_name', pt, 'property_type')
        df_final = df_final.append(df_temp)

In [5]:
df_final = pd.DataFrame()
place_name_list = data.place_name.unique()
prop_type_list = data.property_type.unique()
for pn in place_name_list:
    for pt in prop_type_list:
        df_temp = remove_outlier(data, 0.25, 0.75, 1.5, 'surface_total_in_m2', pn, 'place_name', pt, 'property_type')
        df_final = df_final.append(df_temp)

In [6]:
df_final = pd.DataFrame()
place_name_list = data.place_name.unique()
prop_type_list = data.property_type.unique()
for pn in place_name_list:
    for pt in prop_type_list:
        df_temp = remove_outlier(data, 0.25, 0.75, 1.5, 'surface_covered_in_m2', pn, 'place_name', pt, 'property_type')
        df_final = df_final.append(df_temp)

In [7]:
data = df_final

In [8]:
# armamos un data con los nulos de precio y solo algunas columnas

precio_nulo = data.price_usd_per_m2.isnull()
x = data.loc[precio_nulo,["Id_caso", "place_name", "price_usd_per_m2"]]
x.set_index("Id_caso", inplace = True)

In [9]:
# mediana de precios por barrios

mediana = dict(data.groupby("place_name")["price_usd_per_m2"].median().round(2))

In [10]:
for v, k in mediana.items():
#     print(v, k)
    barrios_nulos = x.place_name == v
    x.loc[barrios_nulos,:] = k

In [11]:
# cantidad de nulos por barrios

data_porc_nulos = pd.DataFrame(x["place_name"].value_counts())
data_porc_nulos.rename(columns = {"place_name":"nulos"}, inplace = True)
data_porc_todos = pd.DataFrame(data.place_name.value_counts())

data_porc_todos
data_porc = pd.concat([data_porc_nulos, data_porc_todos], axis = 1)
data_porc["porc"] = (data_porc.nulos/data_porc.place_name * 100).round(2)
data_porc.sort_values(by = ["porc"], ascending = False)

,nulos,place_name,porc
750.16,8.0,NaN,NaN
1022.73,25.0,NaN,NaN
1100.0,58.0,NaN,NaN
1229.51,2.0,NaN,NaN
1274.34,8.0,NaN,NaN
...,...,...,...
Villa Riachuelo,NaN,5.0,NaN
Villa Santa Rita,NaN,50.0,NaN
Villa Soldati,NaN,14.0,NaN
Villa Urquiza,NaN,1390.0,NaN


In [12]:
# armamos df con los valores imputados para concatenar al data original

a = x.drop("place_name",axis=1)
a.rename(columns = {"price_usd_per_m2": "precio_imputado"}, inplace=True)

In [13]:
a

,precio_imputado
Id_caso,
7137,1466.07
8628,1466.07
9588,1466.07
15836,1466.07
18427,1466.07
...,...
72482,1472.22
30052,1472.22
116358,1472.22


In [14]:
# seteamos índices para concatenar

data.set_index("Id_caso", inplace = True)

In [15]:
data

,property_type,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,price,currency,price_aprox_local_currency,...,surface_total_in_m2,surface_covered_in_m2,cochera,piscina,parrilla,baulera,balcon,terraza,jardin,lavadero
Id_caso,,,,,,,,,,,,,,,,,,,,,
0,PH,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,...,55.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
16,PH,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"-34.6523561177,-58.5016239381",-34.652356,-58.501624,239000.0,USD,4217035.5,...,140.0,98.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
254,PH,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"-34.6572422,-58.5029706",-34.657242,-58.502971,53000.0,USD,935158.5,...,28.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,PH,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"-34.6506036,-58.4961457",-34.650604,-58.496146,245000.0,USD,4322902.5,...,326.0,160.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0
559,PH,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,"-34.6569204,-58.4897843",-34.656920,-58.489784,165000.0,USD,2911342.5,...,150.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22956,PH,|Argentina|Capital Federal|Villa Riachuelo|,Argentina,Capital Federal,"-34.6938592,-58.4670081",-34.693859,-58.467008,150000.0,USD,2646675.0,...,122.0,66.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
119715,PH,|Argentina|Capital Federal|Villa Riachuelo|,Argentina,Capital Federal,"-34.690946,-58.469362",-34.690946,-58.469362,165000.0,USD,2911342.5,...,NaN,20.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0
25766,house,|Argentina|Capital Federal|Villa Riachuelo|,Argentina,Capital Federal,"-34.6918566,-58.4712986",-34.691857,-58.471299,NaN,NaN,NaN,...,372.0,110.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0


In [16]:
data_imputac = pd.concat([data, a], axis = 1)

In [17]:
# imputamos los nulos de mt2 totales con los valores de mt2 cubiertos y lo mismo para las columnas de precio:

super_cub_no_nulo = data.surface_covered_in_m2.notnull()
data.loc[super_cub_no_nulo, "surface_total_in_m2"] = data.loc[super_cub_no_nulo, "surface_covered_in_m2"]

In [18]:
# reemplazamos los valores imputados en la columna precio/mt2

# 'price_per_m2'

precios_no_nulos = data_imputac.precio_imputado.notnull()
data_imputac.loc[precios_no_nulos, "price_usd_per_m2"] = data_imputac.loc[precios_no_nulos, "precio_imputado"]

In [19]:
data = data_imputac

In [20]:
caba_place_name_mask = data.place_name != 'Capital Federal'
surface_total_in_m2_notnull = data.surface_total_in_m2.notnull()
surface_covered_mask_notnull = data.surface_covered_in_m2.notnull()
price_usd_per_m2_mask_notnull = data.price_usd_per_m2.notnull()

In [21]:
data = data.loc[caba_place_name_mask & surface_covered_mask_notnull & surface_total_in_m2_notnull & price_usd_per_m2_mask_notnull]

In [22]:
data.shape

(23107, 36)

In [24]:
property_type_dummies = pd.get_dummies(data.property_type, prefix='prop_type', drop_first = True)

data = pd.concat([data, property_type_dummies], axis=1)

In [25]:
place_name_dummies = pd.get_dummies(data.place_name, prefix='pn', drop_first = True)

data = pd.concat([data, place_name_dummies], axis=1)

In [26]:
data.loc[data.cochera.isnull(), 'cochera'] = 0
data.cochera.astype('uint8')

data.loc[data.piscina.isnull(), 'piscina'] = 0
data.piscina.astype('uint8')

data.loc[data.lavadero.isnull(), 'lavadero'] = 0
data.lavadero.astype('uint8')

data.loc[data.parrilla.isnull(), 'parrilla'] = 0
data.parrilla.astype('uint8')

data.loc[data.terraza.isnull(), 'terraza'] = 0
data.terraza.astype('uint8')

data.loc[data.jardin.isnull(), 'jardin'] = 0
data.jardin.astype('uint8')

data.loc[data.balcon == 'Balcon', 'balcon'] = 1
data.loc[data.balcon == 'BALCON', 'balcon'] = 1
data.loc[data.balcon == 'balcon', 'balcon'] = 1
data.loc[data.balcon.isnull(), 'balcon'] = 0
data.balcon.astype('uint8')

data.loc[data.baulera.isnull(), 'baulera'] = 0
data.baulera.astype('uint8')

Id_caso
0         0
2         0
7         0
8         0
13        0
         ..
121153    0
121157    0
121158    0
121215    0
121217    0
Name: baulera, Length: 23107, dtype: uint8

In [27]:
nulos_piso = data.floor.isnull()
data.loc[nulos_piso, "floor" ] = 4

nulos_rooms = data.rooms.isnull()
data.loc[nulos_rooms, "rooms" ] = 2

In [28]:
erase_columns = ['property_type', 'place_with_parent_names', 'country_name', 'price_per_m2','state_name', 'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'expenses', 'description', 'title', 'Pais', 'Zona', 'Partido_barrio',
       'Localidad', 'Obs_localidad', 'place_name']

In [29]:
data = data.drop(data[erase_columns], axis=1)

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

In [32]:

model = LinearRegression(fit_intercept=True)

feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'cochera', 'piscina',
       'parrilla', 'baulera', 'balcon', 'terraza', 'jardin', 'lavadero',
       'prop_type_apartment', 'prop_type_house', 'prop_type_store',
       'pn_Agronomía', 'pn_Almagro', 'pn_Balvanera', 'pn_Barracas',
       'pn_Barrio Norte', 'pn_Belgrano', 'pn_Boca', 'pn_Boedo', 'pn_Caballito',
       'pn_Centro / Microcentro', 'pn_Chacarita', 'pn_Coghlan',
       'pn_Colegiales', 'pn_Congreso', 'pn_Constitución', 'pn_Flores',
       'pn_Floresta', 'pn_Las Cañitas', 'pn_Liniers', 'pn_Mataderos',
       'pn_Monserrat', 'pn_Monte Castro', 'pn_Nuñez', 'pn_Once', 'pn_Palermo',
       'pn_Palermo Chico', 'pn_Palermo Hollywood', 'pn_Palermo Soho',
       'pn_Palermo Viejo', 'pn_Parque Avellaneda', 'pn_Parque Centenario',
       'pn_Parque Chacabuco', 'pn_Parque Chas', 'pn_Parque Patricios',
       'pn_Paternal', 'pn_Pompeya', 'pn_Puerto Madero', 'pn_Recoleta',
       'pn_Retiro', 'pn_Saavedra', 'pn_San Cristobal', 'pn_San Nicolás',
       'pn_San Telmo', 'pn_Tribunales', 'pn_Velez Sarsfield', 'pn_Versalles',
       'pn_Villa Crespo', 'pn_Villa Devoto', 'pn_Villa General Mitre',
       'pn_Villa Lugano', 'pn_Villa Luro', 'pn_Villa Ortuzar',
       'pn_Villa Pueyrredón', 'pn_Villa Real', 'pn_Villa Riachuelo',
       'pn_Villa Santa Rita', 'pn_Villa Soldati', 'pn_Villa Urquiza',
       'pn_Villa del Parque', 'floor', 'rooms']
X = data[feature_cols]
y = data.price_usd_per_m2

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=1)

scaler = StandardScaler()
scaler.fit_transform(Xtrain)

model.fit(Xtrain, ytrain)
ypred = model.predict(Xtest)
ypred_train = model.predict(Xtrain)

print ('MAE:', mean_absolute_error(ytest, ypred).round(2))
print ('MSE:', mean_squared_error(ytest, ypred).round(2))
print ('RMSE:', np.sqrt(mean_squared_error(ytest, ypred)).round(2))
print ('R2:', r2_score(ytest, ypred).round(2))

MAE: 986.2
MSE: 5455059.28
RMSE: 2335.61
R2: 0.36


In [33]:
r2_score(ytrain, ypred_train).round(2)

0.3